# Stress Level Prediction – LSTM, CNN & NCF Models
Updated notebook including LSTM & CNN deep learning models.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, LSTM, Conv1D, MaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

df = pd.read_csv('dataset_prediksi_stres_200.csv')
df.head()

,employee_id,department,workload,work_life_balance,team_conflict,management_support,work_environment,stress_level
0,EMP0001,HR,9,10,3,5,5,13
1,EMP0002,Finance,8,4,7,6,2,15
2,EMP0003,Finance,10,5,4,7,5,21
3,EMP0004,Marketing,1,10,9,6,1,52
4,EMP0005,IT,1,8,5,7,10,28


## Preprocessing

In [3]:
def classify_level(x):
    if x <= 20: return 0
    elif x <= 40: return 1
    else: return 2

df['label'] = df['stress_level'].apply(classify_level)

le_dept = LabelEncoder()
df['dept_enc'] = le_dept.fit_transform(df['department'])

features = ['workload','work_life_balance','team_conflict','management_support','work_environment']
X = df[features]
y = df['label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
y_train_cat = to_categorical(y_train, 3)
y_test_cat = to_categorical(y_test, 3)

X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

X_train_cnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

df.head()

,employee_id,department,workload,work_life_balance,team_conflict,management_support,work_environment,stress_level,label,dept_enc
0,EMP0001,HR,9,10,3,5,5,13,0,1
1,EMP0002,Finance,8,4,7,6,2,15,0,0
2,EMP0003,Finance,10,5,4,7,5,21,1,0
3,EMP0004,Marketing,1,10,9,6,1,52,2,3
4,EMP0005,IT,1,8,5,7,10,28,1,2


## LSTM Model

In [4]:
lstm_inp = Input(shape=(1, X_train.shape[1]))
x = LSTM(64)(lstm_inp)
x = Dense(32, activation='relu')(x)
out = Dense(3, activation='softmax')(x)

lstm_model = Model(lstm_inp, out)
lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_lstm = lstm_model.fit(X_train_lstm, y_train_cat, epochs=50, batch_size=16, validation_split=0.2, verbose=0)

y_pred_lstm = np.argmax(lstm_model.predict(X_test_lstm), axis=1)
print("LSTM Accuracy:", accuracy_score(y_test, y_pred_lstm))
print(classification_report(y_test, y_pred_lstm))

E0000 00:00:1763878661.778885  396342 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1763878661.813927  396342 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
LSTM Accuracy: 0.55
              precision    recall  f1-score   support

           0       0.33      0.14      0.20         7
           1       0.17      0.11      0.13         9
           2       0.65      0.83      0.73        24

    accuracy                           0.55        40
   macro avg       0.38      0.36      0.35        40
weighted avg       0.48      0.55      0.50        40



## CNN Model

In [5]:
cnn_inp = Input(shape=(X_train.shape[1], 1))
x = Conv1D(32, kernel_size=2, activation='relu')(cnn_inp)
x = MaxPooling1D()(x)
x = Conv1D(64, kernel_size=2, activation='relu')(x)
x = GlobalAveragePooling1D()(x)
x = Dense(32, activation='relu')(x)
out = Dense(3, activation='softmax')(x)

cnn_model = Model(cnn_inp, out)
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_cnn = cnn_model.fit(X_train_cnn, y_train_cat, epochs=50, batch_size=16, validation_split=0.2, verbose=0)

y_pred_cnn = np.argmax(cnn_model.predict(X_test_cnn), axis=1)
print("CNN Accuracy:", accuracy_score(y_test, y_pred_cnn))
print(classification_report(y_test, y_pred_cnn))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
CNN Accuracy: 0.475
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.22      0.22      0.22         9
           2       0.59      0.71      0.64        24

    accuracy                           0.47        40
   macro avg       0.27      0.31      0.29        40
weighted avg       0.40      0.47      0.43        40



## Neural Collaborative Filtering (NCF) Model

In [6]:
user_enc = LabelEncoder()
df['user_enc'] = user_enc.fit_transform(df['employee_id'])
num_users = df['user_enc'].nunique()
num_items = df['dept_enc'].nunique()

X_user = df['user_enc']
X_item = df['dept_enc']
y2 = df['label']

X_user_train, X_user_test, X_item_train, X_item_test, y2_train, y2_test = train_test_split(
    X_user, X_item, y2, test_size=0.2, random_state=42)

y2_train_cat = to_categorical(y2_train, 3)
y2_test_cat = to_categorical(y2_test, 3)

user_in = Input(shape=(1,))
item_in = Input(shape=(1,))

user_emb = Embedding(num_users, 16)(user_in)
item_emb = Embedding(num_items, 16)(item_in)

u = Flatten()(user_emb)
i = Flatten()(item_emb)

concat = Concatenate()([u, i])
x = Dense(32, activation='relu')(concat)
x = Dense(16, activation='relu')(x)
out = Dense(3, activation='softmax')(x)

ncf = Model([user_in, item_in], out)
ncf.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

ncf.fit([X_user_train, X_item_train], y2_train_cat, epochs=50, batch_size=16, verbose=0)

y2_pred = np.argmax(ncf.predict([X_user_test, X_item_test]), axis=1)
print("NCF Accuracy:", accuracy_score(y2_test, y2_pred))
print(classification_report(y2_test, y2_pred))

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7f9150241760> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
NCF Accuracy: 0.5
              precision    recall  f1-score   support

           0       0.14      0.14      0.14         7
           1       0.00      0.00      0.00         9
           2       0.